<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20Scikit-learn.png?raw=true">
</p>

 # **<font color="DeepPink">M√©tricas de evaluaci√≥n del desempe√±o: Clustering</font>**

<p align="justify">
En el contexto de clustering, las m√©tricas de evaluaci√≥n se utilizan para medir qu√© tan bien se agrupan los datos. Estas m√©tricas permiten evaluar la calidad de los clusters generados por el algoritmo de clustering y seleccionar el n√∫mero √≥ptimo de clusters, si es necesario.
<br><br>
Algunas de las m√©tricas m√°s comunes utilizadas en clustering son el puntaje de Silueta (<b>Silhouette Score</b>), el √≠ndice de Davies-Bouldin (<b>Davies-Bouldin Index</b>), la inercia (<b>Inertia</b>) y el √≠ndice de Calinski-Harabasz (<b>Calinski-Harabasz Index</b>).
<br><br>
Es importante destacar que no hay una m√©trica √∫nica y universalmente adecuada para todos los conjuntos de datos y problemas de clustering. La elecci√≥n de la m√©trica depender√° de la naturaleza de los datos, la forma en que se agrupan los datos y el objetivo espec√≠fico que se busca alcanzar con el an√°lisis de clustering. Adem√°s, es com√∫n utilizar m√∫ltiples m√©tricas para tener una evaluaci√≥n m√°s completa del rendimiento del algoritmo de clustering y la calidad de los clusters generados.


 # **<font color="DeepPink">Carga de las librer√≠as b√°sicas</font>**

In [4]:
import numpy as np
import pandas as pd

In [5]:
import plotly.express as px
import plotly.graph_objects as go

 # **<font color="DeepPink">Conjunto de datos</font>**

In [6]:
url = "https://raw.githubusercontent.com/cristiandarioortegayubro/BA/main/Datasets/clientes_mall.csv"

In [7]:
datos = pd.read_csv(url)

In [8]:
datos.head()

CustomerID  Gender  Age  AnnualIncome  SpendingScore
0           1    Male   19            15             39
1           2    Male   21            15             81
2           3  Female   20            16              6
3           4  Female   23            16             77
4           5  Female   31            17             40

<p align="justify">
Este conjuntod de datos contiene informaci√≥n sobre las persona que han visitado un centro comercial. Contiene las siguientes variables: <code>CustomerID</code>, <code>Gender</code>, <code>Age</code>, <code>AnnualIncome</code> y <code>SpendingScore</code> (puntaje asignado por el centro comercial basado en el comportamiento del cliente y la naturaleza del gasto).


In [9]:
datos = datos.drop(columns=["CustomerID", "Gender"])                            #se eliminan las variable no relevantes
datos

Age  AnnualIncome  SpendingScore
0     19            15             39
1     21            15             81
2     20            16              6
3     23            16             77
4     31            17             40
..   ...           ...            ...
195   35           120             79
196   45           126             28
197   32           126             74
198   32           137             18
199   30           137             83

[200 rows x 3 columns]

In [10]:
# Renombramos columnas
datos.rename(columns={"Age": "Edad",
                      "AnnualIncome":"IngresoAnual",
                      "SpendingScore":"ScoreGasto"},inplace=True)

In [11]:
datos.head()

Edad  IngresoAnual  ScoreGasto
0    19            15          39
1    21            15          81
2    20            16           6
3    23            16          77
4    31            17          40

 ## **<font color="DeepPink">Escalado de datos</font>**

<p align="justify">
En los modelos que se basan en la distancia, por ejemplo K-means, las variables deben ser escaladas para que cada una contribuya aproximadamente por igual a los c√°lculos de distancia. Escalar y centrar las variables para que tengan media 0 y desviaci√≥n est√°ndar 1, asegura que todas las variables tengan el mismo peso cuando se realice el clustering.

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [13]:
data_scaled = scaler.fit_transform(datos)
data_scaled[:10]

array([[-1.42456879, -1.73899919, -0.43480148],
       [-1.28103541, -1.73899919,  1.19570407],
       [-1.3528021 , -1.70082976, -1.71591298],
       [-1.13750203, -1.70082976,  1.04041783],
       [-0.56336851, -1.66266033, -0.39597992],
       [-1.20926872, -1.66266033,  1.00159627],
       [-0.27630176, -1.62449091, -1.71591298],
       [-1.13750203, -1.62449091,  1.70038436],
       [ 1.80493225, -1.58632148, -1.83237767],
       [-0.6351352 , -1.58632148,  0.84631002]])

In [14]:
data_scaled = pd.DataFrame(data_scaled, columns = datos.columns)
data_scaled.describe().T.round(2)

count  mean  std   min   25%   50%   75%   max
Edad          200.0  -0.0  1.0 -1.50 -0.72 -0.20  0.73  2.24
IngresoAnual  200.0  -0.0  1.0 -1.74 -0.73  0.04  0.67  2.92
ScoreGasto    200.0  -0.0  1.0 -1.91 -0.60 -0.01  0.89  1.89

In [15]:
data_scaled.head()

Edad  IngresoAnual  ScoreGasto
0 -1.424569     -1.738999   -0.434801
1 -1.281035     -1.738999    1.195704
2 -1.352802     -1.700830   -1.715913
3 -1.137502     -1.700830    1.040418
4 -0.563369     -1.662660   -0.395980

 # **<font color="DeepPink">K-means</font>**

<p align="justify">
Un cluster hace referencia a un conjunto de datos (utilizando solo vectores
de entrada) agrupados en base a ciertas similitudes. K-means es el algoritmo m√°s popular y simple de aprendizaje autom√°tico no supervisado. Se basa en el concepto de distancia e identifica $K$ n√∫mero de centroides y luego asigna los puntos de datos al cluster m√°s cercano.

<p align="justify">
Existen 2 conceptos fundamentales para comprender el concepto de clustering y su evaluaci√≥n:
<ul align="justify">
<li><b>Cohesi√≥n</b>: el miembro de cada cl√∫ster debe ser lo m√°s cercano posible a los otros miembros del mismo cl√∫ster. (Minimizar la distancia intra-cluster)
<li><b>Separaci√≥n</b>: Los cl√∫ster deben estar ampliamente separados entre ellos. (Maximizar la distancia inter-cluster)

In [16]:
from sklearn.cluster import KMeans
from sklearn import metrics

<p align="justify">
Con la clase <code>KMeans</code> del m√≥dulo <code>cluster</code> de <code>Scikit-Learn</code> se pueden entrenar modelos de clustering utilizando el algoritmo K-means. Puede encontrarse una descripci√≥n detallada de todos los hiperpar√°metros en <a href="https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html">sklearn.cluster.KMeans</a>. Sin embargo, entre sus par√°metros destacan:
<br><br>
<ul align="justify">
<li><code>n_clusters</code>: determina el n√∫mero $K$ de clusters que se van a generar.
<br><br>
<li><code>init</code>: estrategia para asignar los centroides iniciales. Por defecto se emplea <code>k-means++</code>, una estrategia que trata de alejar los centroides lo m√°ximo posible facilitando la convergencia. Sin embargo, esta estrategia puede ralentizar el proceso cuando hay muchos datos, si esto ocurre, es mejor utilizar <code>random</code>.
<br><br>
<li><code>n_init</code>: determina el n√∫mero de veces que se va a repetir el proceso, cada vez con una asignaci√≥n aleatoria inicial distinta. Es recomendable que este √∫ltimo valor sea alto, entre 10-25.
<br><br>
<li><code>max_iter</code>: n√∫mero m√°ximo de iteraciones permitidas.
<br><br>
<li><code>random_state</code>: semilla para garantizar la reproducibilidad de los resultados.

In [17]:
model_kmeans = KMeans(n_clusters = 5, n_init = 25, random_state = 123)
model_kmeans.fit(datos)

KMeans(n_clusters=5, n_init=25, random_state=123)

In [18]:
prediction = model_kmeans.predict(datos)
prediction[:10]

array([4, 3, 4, 3, 4, 3, 4, 3, 4, 3], dtype=int32)

In [19]:
centroids = model_kmeans.cluster_centers_
labels = model_kmeans.labels_

In [20]:
centroids = pd.DataFrame(centroids, columns=datos.columns)
centroids

Edad  IngresoAnual  ScoreGasto
0  40.324324     87.432432   18.189189
1  43.282051     55.025641   49.692308
2  32.692308     86.538462   82.128205
3  25.521739     26.304348   78.565217
4  45.217391     26.304348   20.913043

In [21]:
labels

array([4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3,
       4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3, 4, 3,
       4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 2, 1, 2, 0, 2, 0, 2,
       0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2,
       0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2,
       0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2,
       0, 2], dtype=int32)

In [22]:
datos['Cluster'] = labels
datos.head()

Edad  IngresoAnual  ScoreGasto  Cluster
0    19            15          39        4
1    21            15          81        3
2    20            16           6        4
3    23            16          77        3
4    31            17          40        4

In [23]:
fig = go.Figure([go.Scatter3d(x = datos.Edad,
                              y = datos.IngresoAnual,
                              z = datos.ScoreGasto,
                              mode = "markers",
                              name = "Clusters",
                              marker = dict(color = datos.Cluster)),

                 go.Scatter3d(x = centroids.Edad,
                              y = centroids.IngresoAnual,
                              z = centroids.ScoreGasto,
                              mode = "markers",
                              name = "Centroide",
                              marker_color = "red",
                              marker = dict(size = 12)),
                 ])

fig.update_layout(title = "Edad, Ingresos Anuales y Score de Gasto")

fig.show()

 ## **<font color="DeepPink">Kmeans_plusplus</font>**

In [38]:
from sklearn.cluster import kmeans_plusplus
import numpy as np

In [42]:
sample_weight = _check_sample_weight(sample_weight, X, dtype=X.dtype)

NameError: name '_check_sample_weight' is not defined

In [43]:
from sklearn.cluster import kmeans_plusplus
import numpy as np
from sklearn.cluster import KMeans

# Convert the relevant columns of the DataFrame to a NumPy array
X = datos[['Edad', 'IngresoAnual', 'ScoreGasto']].values

# Now use the NumPy array as input to kmeans_plusplus
centers, indices = kmeans_plusplus(X, n_clusters=5, random_state=123)

# Initialize and fit the KMeans model using the centers from kmeans_plusplus
model_kmeans_plus = KMeans(n_clusters=5, init=centers, n_init=1, random_state=123)
model_kmeans_plus.fit(X)  # Fit the KMeans model using the NumPy array

# Get the cluster labels
labels = model_kmeans_plus.labels_

In [52]:
centers = model_kmeans_plus.cluster_centers_
labels2 = model_kmeans_plus.labels_

In [53]:
centers

array([[32.69230769, 86.53846154, 82.12820513],
       [44.31818182, 25.77272727, 20.27272727],
       [40.39473684, 87.        , 18.63157895],
       [24.8       , 41.46      , 63.7       ],
       [53.82352941, 54.7254902 , 48.98039216]])

In [54]:
labels2

array([1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3,
       1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 1, 3, 4, 3, 1, 3,
       1, 3, 4, 3, 3, 3, 4, 3, 3, 4, 4, 4, 4, 4, 3, 4, 4, 3, 4, 4, 4, 3,
       4, 4, 3, 3, 4, 4, 4, 4, 4, 3, 4, 4, 3, 4, 4, 4, 4, 4, 3, 4, 4, 3,
       3, 4, 4, 3, 4, 4, 4, 3, 4, 3, 4, 3, 3, 4, 4, 3, 4, 3, 4, 4, 4, 4,
       4, 3, 4, 3, 3, 3, 4, 4, 4, 4, 3, 4, 4, 0, 2, 0, 2, 0, 2, 0, 2, 0,
       2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0,
       2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0,
       2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0,
       2, 0], dtype=int32)

In [49]:
centers = pd.DataFrame(centers, columns=['Edad', 'IngresoAnual', 'ScoreGasto']) # Only use the relevant columns
centers

Edad  IngresoAnual  ScoreGasto
0  32.692308     86.538462   82.128205
1  44.318182     25.772727   20.272727
2  40.394737     87.000000   18.631579
3  24.800000     41.460000   63.700000
4  53.823529     54.725490   48.980392

In [45]:
indices

array([139,  28, 170,  58,  73])

 ## **<font color="DeepPink">Evaluaci√≥n del modelo</font>**

<p align="justify">
Se puede evaluar la calidad de los modelos de clustering con las siguientes m√©tricas:



### **<font color="DeepPink">Inercia (Inertia)**

<p align="justify">
La inercia mide la suma de las distancias cuadr√°ticas de cada uno de los puntos de datos a sus centroide m√°s cercano. Un valor m√°s bajo de inercia indica clusters m√°s densos y compactos. Su f√≥rmula es:

$$Inertia=\sum_{k=1}^K\sum_{i=1}^{n_k}||d_i-c_k||^2$$


In [50]:
inertia = model_kmeans.inertia_
inertia

28145.181554863335

In [51]:
inertia = model_kmeans_plus.inertia_
inertia

79276.7490465984

### **<font color="DeepPink">Puntuaci√≥n de Silueta (Silhouette Score)**

<p align="justify">
Esta m√©trica cuantifica qu√© tan bien un punto se ajusta a su propio cluster en comparaci√≥n con los clusters vecinos m√°s cercanos. El valor de silueta var√≠a entre -1 y 1, donde un valor m√°s cercano a 1 indica que el punto est√° bien asignado a su cluster, mientras que un valor cercano a -1 indica que el punto podr√≠a pertenecer a un cluster diferente. Un valor cercano a 0 sugiere que el punto est√° cerca del l√≠mite entre dos clusters. Su f√≥rmula es:

$$s_i = \frac{b_i - a_i}{max(a_i, b_i)}$$

<br><p align="justify">
donde:
<br><ul align="justify">
<li>$a_i$ es la distancia media entre el punto de datos $i$ y todos los dem√°s puntos en el mismo cluster al que pertenece.
<li>$b_i$ es la distancia media entre el punto de datos $i$ y todos los puntos en el cluster m√°s cercano diferente al que pertenece $i$.

<p align="justify">
Para calcular la Silueta Promedio (Average Silhouette), se promedia la puntuaci√≥n de silueta de todos los puntos de datos en el conjunto de datos:

$$SS = \frac{\sum_{i=1}^n s_i}{n}$$




In [25]:
ss = metrics.silhouette_score(datos, labels)
ss.round(2)

0.44

In [55]:
ss = metrics.silhouette_score(datos, labels2)
ss.round(2)

0.43

Una puntuaci√≥n de silueta de 0.44 indica que las agrupaciones tienen una separaci√≥n moderada y que los puntos dentro de cada cluster est√°n relativamente bien agrupados, pero podr√≠a haber alguna superposici√≥n o ambig√ºedad en la separaci√≥n de algunos clusters.

### **<font color="DeepPink">√≠ndice de Calinski-Harabasz (Calinski-Harabasz Index)**

<p align="justify">
Tambi√©n conocido como el √≠ndice de distancias entre los clusters, es una m√©trica que relaciona la separaci√≥n (distancia inter-cluster) y la cohesi√≥n (distancia intra-cluster). Un valor m√°s alto del √≠ndice de Calinski-Harabasz indica clusters m√°s densos y bien separados. Su f√≥rmula es:

$$CH=\frac{\frac{\sum_{k=1}^K{n_k}||c_k-c||^2}{K-1}}{\frac{\sum_{k=1}^K\sum_{i=1}^{n_k}||d_i-c_k||^2}{N-K}}$$




In [26]:
metrics.calinski_harabasz_score(datos, labels)

151.18468575912456

In [56]:
metrics.calinski_harabasz_score(datos, labels2)

141.19560608192054

 ## **<font color="DeepPink">N√∫mero √≥ptimo de clusters</font>**

<p align="justify">
Determinar el n√∫mero √≥ptimo de clusters es uno de los pasos m√°s importantes a la hora de aplicar m√©todos de clustering, sobre todo cuando se trata del algoritmo K-means, donde el n√∫mero se tiene que especificar antes de entrenar el modelo.

 ### **<font color="DeepPink">M√©todo del codo</font>**

<p align="justify">
El m√©todo del codo, tambi√©n conocido como Elbow, sigue una estrategia com√∫nmente empleada para encontrar el valor √≥ptimo de un hiperpar√°metro. La idea es probar un rango de valores del hiperpar√°metro en cuesti√≥n (<code>n_clusters</code>), representar gr√°ficamente los resultados obtenidos con cada uno, e identificar aquel punto de la curva (codo) a partir del cual la mejora deja de ser notable.
<br><br>
Una forma sencilla de estimar el n√∫mero $K$ √≥ptimo de clusters, es aplicar el algoritmo de K-means para un rango de valores de $K$ e identificar aquel valor a partir del cual la reducci√≥n en la suma total de distancia intra-cluster (inertia) deja de ser sustancial. A esta estrategia se la conoce como m√©todo del codo o elbow method.



In [27]:
inertia = []
k_range = range(1, 10)

In [28]:
for k in k_range:
   model_kmeans = KMeans(n_clusters=k,
                         n_init = 'auto',
                         random_state=123).fit(datos)
   inertia.append(model_kmeans.inertia_)

In [29]:
clusters = pd.DataFrame({'clusters':k_range,
                         'inertia':inertia})

In [30]:
fig = px.line(clusters,
              x = "clusters",
              y = "inertia",
              markers = True,
              title = "Metodo del codo",
              template = "gridon")
fig.show()

In [57]:
inertia2 = []
k_range = range(1, 10)

In [59]:
from sklearn.cluster import KMeans

inertia2 = []
k_range = range(1, 10)

for k in k_range:
    # Use KMeans class with kmeans_plusplus initialization
    model_kmeans = KMeans(n_clusters=k,
                         init='k-means++',  # Use kmeans++ for initialization
                         n_init='auto',
                         random_state=123).fit(datos)
    # Append the inertia from the fitted model
    inertia2.append(model_kmeans.inertia_)

In [61]:
clusters2 = pd.DataFrame({'clusters':k_range,
                         'inertia':inertia2})

In [62]:
fig = px.line(clusters2,
              x = "clusters",
              y = "inertia",
              markers = True,
              title = "Metodo del codo",
              template = "gridon")
fig.show()

 ### **<font color="DeepPink">M√©todo silhuette</font>**

<p align="justify">
El m√©todo de average silhouette considera como n√∫mero √≥ptimo de clusters aquel que maximiza la media del valor de silueta de todas las observaciones.

In [31]:
silhouette = []
k_clusters = range(2, 15)

In [32]:
for k in k_clusters:
    model_kmeans = KMeans(n_clusters   = k,
                          n_init       = 20,
                          random_state = 123)

    cluster_labels = model_kmeans.fit_predict(datos)
    silhouette_avg = metrics.silhouette_score(datos, cluster_labels)
    silhouette.append(silhouette_avg)

In [33]:
fig = px.line(x = k_clusters,
              y = silhouette,
              markers = True,
              title="Metodo silhouette",
              template="gridon",
              labels = {"x":"clusters", "y":"silhuette score"})
fig.show()

<p align="justify">
El valor medio de las siluetas se maximiza con 6 clusters. Acorde a este criterio, K = 6 es la mejor opci√≥n.
<br><br>
Ambos criterios, elbow y silhouette, identifican el valor K = 6 como valor √≥ptimo de clusters.

 # **<font color="DeepPink">Conclusiones</font>**

<p align="justify">
üëÄ En este colab nosotros:<br><br>
‚úÖ Realizamos el entrenamiento de un algoritmo de clustering. <br>
‚úÖ Calculamos las m√©tricas del modelo, para determinar la calidad de las agrupaciones. <br>
‚úÖ Determinamos el n√∫mero √≥ptimo de clusters. <br>

<p align="justify">



<br>
<br>
<p align="center"><b>
üíó
<font color="DeepPink">
Hemos llegado al final de nuestro colab, a seguir codeando...
</font>
</p>
